In [7]:
import sys
print(sys.path)
import re
import os
import csv
import nltk
import sklearn

import numpy as np
import pandas as pd
import tensorflow as tf

from gensim.models import KeyedVectors

from nltk.corpus import stopwords

EMBEDDING_FILE = '/Users/swapnil/work/Kaggle/data/googleNewsEmbeddings/GoogleNews-vectors-negative300.bin'
TRAIN_FILE = '/Users/swapnil/work/Kaggle/data/quoraPairs/train.csv'
TEST_FILE = '/Users/swapnil/work/Kaggle/data/quoraPairs/test.csv'
EMBEDDING_DIM = 300

print('tf version ',tf.__version__)


['', '/Users/swapnil/anaconda2/lib/python27.zip', '/Users/swapnil/anaconda2/lib/python2.7', '/Users/swapnil/anaconda2/lib/python2.7/plat-darwin', '/Users/swapnil/anaconda2/lib/python2.7/plat-mac', '/Users/swapnil/anaconda2/lib/python2.7/plat-mac/lib-scriptpackages', '/Users/swapnil/anaconda2/lib/python2.7/lib-tk', '/Users/swapnil/anaconda2/lib/python2.7/lib-old', '/Users/swapnil/anaconda2/lib/python2.7/lib-dynload', '/Users/swapnil/anaconda2/lib/python2.7/site-packages', '/Users/swapnil/anaconda2/lib/python2.7/site-packages/Sphinx-1.5.1-py2.7.egg', '/Users/swapnil/anaconda2/lib/python2.7/site-packages/aeosa', '/Users/swapnil/anaconda2/lib/python2.7/site-packages/IPython/extensions', '/Users/swapnil/.ipython']
('tf version ', '1.0.0')


In [8]:
word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, \
                                            binary=True)

trainFullData = pd.read_csv(TRAIN_FILE)

In [3]:
test = pd.read_csv(TEST_FILE)

In [17]:
print('Total rows ',trainFullData.shape[0])
train = trainFullData.sample(frac=1.0,random_state=1988)
valid = train.iloc[range(1,train.shape[0]/80),:]
print('Valid rows ',valid.shape[0])
train = train.iloc[range(train.shape[0]/80,train.shape[0]),:]
print('Train rows ',train.shape[0])

print(train.iloc[range(1,5),:])
print('Found %s word vectors of word2vec' % len(word2vec.vocab))
print('train value counts ',train['is_duplicate'].value_counts())
print('valid value counts ',valid['is_duplicate'].value_counts())

('Total rows ', 404290)
('Valid rows ', 5052)
('Train rows ', 399237)
            id    qid1    qid2  \
304110  304110   78351  367254   
385709  385709  490187  408229   
112546  112546  131684  184123   
21116    21116   39783   39784   

                                                question1  \
304110              How do I overcome my extreme anxiety?   
385709                  How should I focus on my studies?   
112546  What is the use of having flavors in the condoms?   
21116                   What are the theories of poverty?   

                                      question2  is_duplicate  
304110     How do you cope with social anxiety?             1  
385709                    How I focus on study?             1  
112546    What is the use of flavoured condoms?             1  
21116   Is telepathy real? What is the process?             0  
Found 3000000 word vectors of word2vec
('train value counts ', 0    251845
1    147392
Name: is_duplicate, dtype: int64)
('valid valu

In [35]:
MAX_SEQ_LEN = 200
PRINT_STATUS_ITER = 50000
stops = set(stopwords.words("english"))
print('Test print')
buckets = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]

def cleanText(text):
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", str(text))
    return text

class PairScore:
    def __init__(self,ind1,ind2,dist):
        self.ind1 = ind1
        self.ind2 = ind2
        self.dist = dist
    
    def __lt__(self,other):
        return self.dist > other.dist

class Pair:
    def __init__(self,vec1,vec2,ind1,ind2):
        self.vec1 = vec1
        self.vec2 = vec2
        self.ind1 = ind1
        self.ind2 = ind2

class DataInstance:
    def __init__(self,dataId,pairs,label):
        self.dataId = dataId
        self.pairs = pairs
        self.label = label
    
    
def generateClosePairs(embedding1,embedding2,smallQuestion,bigQuestion):
    listPairs = []
    # Creating all possible pairs.
    for i in range(0,embedding1.shape[0]):
        for j in range(0,embedding2.shape[0]):
            p = PairScore(i,j,sklearn.metrics.pairwise
                          .cosine_similarity(embedding1[i].reshape(1,-1),embedding2[j].reshape(1,-1))[0])
            listPairs.append(p)
    
    sortedPairs = sorted(listPairs)
    
    # Creating pairs by finding closest word to the word from shorter question.
    smallConsidered = [False for i in range(embedding1.shape[0])]
    bigConsidered = [False for i in range(embedding2.shape[0])]
    finalPairs = []
    for pair in sortedPairs:
        if smallConsidered[pair.ind1] == False:
            finalPair = Pair(embedding1[pair.ind1],embedding2[pair.ind2],pair.ind1,pair.ind2)
            finalPairs.append(finalPair)
            smallConsidered[pair.ind1] = True
            bigConsidered[pair.ind2] = True
            
    for i,considered in enumerate(bigConsidered):
        if considered == False:
            finalPair = Pair(np.zeros(EMBEDDING_DIM),embedding2[i],-1,i)
            finalPairs.append(finalPair)
            
    return finalPairs
        
    #for finalPair in finalPairs:
     #   print(smallQuestion[finalPair.ind1] if finalPair.ind1!=-1 else "-1",",",bigQuestion[finalPair.ind2])
    
def processData(data,isTrain):
    processedList = []
    maxSeqLen = 0
    indRow = 1
    for _,row in data.iterrows():
        question1 = cleanText(row['question1'])
        question2 = cleanText(row['question2'])
        if isTrain == True:
            label = row['is_duplicate']
        else:
            label = None
          
        if isTrain == True:
            dataId = row['id']
        else:
            dataId = row['test_id']
    
        question1Splits = question1.lower().split()
        question2Splits = question2.lower().split()

        # Removing stopwords.
        shortQuestion1 = [w for w in question1Splits if not w in stops]
        shortQuestion2 = [w for w in question2Splits if not w in stops]

        # Deciding short question and long question.
        if len(shortQuestion1) < len(shortQuestion2):
            smallQuestion = shortQuestion1
            bigQuestion = shortQuestion2
        else:
            smallQuestion = shortQuestion2
            bigQuestion = shortQuestion1
        
        smallEmbedding = np.zeros((len(smallQuestion), EMBEDDING_DIM))
        bigEmbedding = np.zeros((len(bigQuestion), EMBEDDING_DIM))
    
        # Getting embedding vectors for each word.
        wordsBelowScore = np.zeros(len(buckets))
    
        for i in range(0,len(smallQuestion)):
            word = smallQuestion[i]
            if word in word2vec.vocab:
                smallEmbedding[i] = word2vec.word_vec(word)
    
        for i in range(0,len(bigQuestion)):
            word = bigQuestion[i]
            if word in word2vec.vocab:
                bigEmbedding[i] = word2vec.word_vec(word)
        
        pairs = generateClosePairs(smallEmbedding,bigEmbedding,smallQuestion,bigQuestion)
        processedList.append(DataInstance(dataId,pairs,label))
        if len(pairs) > maxSeqLen:
            maxSeqLen = len(pairs)
            
    return maxSeqLen,processedList

def createDataset(processedList):
    dataset = np.zeros((len(processedList),MAX_SEQ_LEN,EMBEDDING_DIM))
    lenthSet = np.zeros(len(processedList))
    labels = np.zeros(len(processedList))
    dataIds = np.zeros(len(processedList))
    for i,processed in enumerate(processedList):
        pairs = processed.pairs
        j = 0;
        for pair in pairs:
            diffSqr = np.square(np.subtract(pair.vec1,pair.vec2))
            dataset[i,j,:] = diffSqr
            j = j + 1;
        while j < MAX_SEQ_LEN:
            dataset[i,j,:] = -1 * np.ones(EMBEDDING_DIM)
            j = j + 1
        lenthSet[i] = len(pairs)
        labels[i] = processed.label
        dataIds[i] = processed.dataId
    return dataset,lenthSet,labels,dataIds

Test print


In [6]:
#trainMaxSeqLen,trainProcessedList = processData(train,True)    

In [7]:
#np.save('/Users/swapnil/work/Kaggle/data/quoraPairs/processedData/trainProcessedList.npy',trainProcessedList)
#np.save('/Users/swapnil/work/Kaggle/data/quoraPairs/processedData/trainMaxSeqLen.npy',trainMaxSeqLen)

In [8]:
#trainProcessedList = np.load('/Users/swapnil/work/Kaggle/data/quoraPairs/processedData/trainProcessedList.npy')
#trainMaxSeqLen = np.load('/Users/swapnil/work/Kaggle/data/quoraPairs/processedData/trainMaxSeqLen.npy')

In [9]:
#print(trainProcessedList.shape)

In [10]:
#trainDataset,trainLengthSet,trainLabels,trainDataIds = createDataset(trainProcessedList,trainMaxSeqLen)

In [11]:
#np.save('/Users/swapnil/work/Kaggle/data/quoraPairs/processedData/trainDataset.npy',trainDataset)
#np.save('/Users/swapnil/work/Kaggle/data/quoraPairs/processedData/trainLengthSet.npy',trainLengthSet)
#np.save('/Users/swapnil/work/Kaggle/data/quoraPairs/processedData/trainLabels.npy',trainLabels)
#np.save('/Users/swapnil/work/Kaggle/data/quoraPairs/processedData/trainDataIds.npy',trainDataIds)

In [18]:
validSeqLen,processedValidList = processData(valid,True)
validData,validSeqLen,validLabels,_ = createDataset(processedValidList)

Start of processData
Start of createDataset
('createDataset curr iter ', 0)


In [19]:
print(validData.shape)

(5052, 200, 300)


In [36]:
BATCH_SIZE = 50
STATE_SIZE = 100

tf.reset_default_graph()
graph = tf.Graph()
with graph.as_default():
    
    batchDataTensor = tf.placeholder(tf.float32, [BATCH_SIZE, MAX_SEQ_LEN, EMBEDDING_DIM])
    batchSeqLengthTensor = tf.placeholder(tf.int32,[BATCH_SIZE])
    batchLabelsTensor = tf.placeholder(tf.float32,[BATCH_SIZE])
    
    validDataTensor = tf.placeholder(tf.float32,shape=validData.shape)
    validSeqLengthTensor = tf.placeholder(tf.int32, shape=validSeqLen.shape)
    validLabelsTensor = tf.placeholder(tf.float32,[validData.shape[0]])
    
    #logisticWeights = tf.Variable(tf.truncated_normal([STATE_SIZE,1], stddev=0.1))
    #bias = tf.Variable(tf.truncated_normal([1], stddev=0.1))
    
    #testBatchData = tf.placeholder(tf.float32, [None, maxSeqLen, EMBEDDING_DIM])
    #testBatchSeqLength = tf.placeholder(tf.int32,[tf.shape(batchData)[0]])
    
    
    #init_state = tf.get_variable('init_state', [1, STATE_SIZE],
                                 #initializer=tf.constant_initializer(0.0))
    #init_state = tf.tile(init_state, [BATCH_SIZE, 1])
    
    #init_state = tf.get_variable('init_state', [BATCH_SIZE, STATE_SIZE],initializer=tf.constant_initializer(0.0))
    
    init_state = tf.contrib.rnn.LSTMStateTuple(tf.zeros([BATCH_SIZE,STATE_SIZE]),tf.zeros([BATCH_SIZE,STATE_SIZE]))
    
    def getLogits(rnnOutputs,modelSeqLength):
        logisticWeights = tf.get_variable('logisticWeights',initializer=tf.truncated_normal([STATE_SIZE,1], stddev=0.1))
        bias = tf.get_variable('logisticBias',initializer=tf.truncated_normal([1], stddev=0.1))
        lastIndex = tf.range(tf.shape(rnnOutputs)[0])*tf.shape(rnnOutputs)[1] + (modelSeqLength - 1)
        lastOutput = tf.gather(tf.reshape(rnnOutputs, [-1, STATE_SIZE]), lastIndex)
        logits = tf.matmul(lastOutput,logisticWeights) + bias
        return logits
    
    def model(modelData,modelSeqLength):
        rnnCell = tf.contrib.rnn.BasicLSTMCell(STATE_SIZE,state_is_tuple=True)
        rnnOutputs, state = tf.nn.dynamic_rnn(rnnCell, modelData, dtype=tf.float32,initial_state=init_state)
        return getLogits(rnnOutputs,modelSeqLength)
        
    
    with tf.variable_scope('train') as scope:
        logits = model(batchDataTensor,batchSeqLengthTensor)
        loss = tf.reduce_mean(
        tf.nn.sigmoid_cross_entropy_with_logits(logits=logits[:,0], labels=batchLabelsTensor))   
        optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

        batchPreds = tf.nn.sigmoid(logits[:,0])
        batchLogLoss = tf.losses.log_loss(batchLabelsTensor, batchPreds)
        scope.reuse_variables()
        rnnCell = tf.contrib.rnn.BasicLSTMCell(STATE_SIZE,state_is_tuple=True)
        rnnOutputs, state = tf.nn.dynamic_rnn(rnnCell, validDataTensor, dtype=tf.float32)
        validLogits = getLogits(rnnOutputs,validSeqLengthTensor)
        #validLogits = model(validDataTensor,validSeqLengthTensor)
        validPreds = tf.nn.sigmoid(validLogits[:,0])
        validLogLoss = tf.losses.log_loss(validLabelsTensor, validPreds)
    #testPreds = tf.nn.sigmoid(model(testBatchData,testBatchSeqLength))


In [ ]:
def accuracy(preds,labels):
    predClass = np.where(preds > 0.5,np.ones(len(preds)),np.zeros(len(preds)))
    return float(sum(predClass == labels)) / len(preds)


num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  saver = tf.train.Saver()
  for step in range(num_steps):
    offset = (step * BATCH_SIZE) % (train.shape[0] - BATCH_SIZE)
    batchDataFrame = train.iloc[range(offset,(offset + BATCH_SIZE)),:]
    batchSeqLen,processedBatchList = processData(batchDataFrame,True)
    batchData,batchSeqLen,batchLabels,_ = createDataset(processedBatchList)
    
    feed_dict = {batchDataTensor : batchData, batchSeqLengthTensor : batchSeqLen, validDataTensor:validData, 
                validSeqLengthTensor:validSeqLen, batchLabelsTensor:batchLabels, validLabelsTensor:validLabels}
    _, lossVal, batchPredsVal,validPredsVal,batchLogLossVal,validLogLossVal= session.run(
      [optimizer, loss, batchPreds,validPreds,batchLogLoss,validLogLoss], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, lossVal))
      print('Minibatch accuracy: %.3f%%' % accuracy(batchPredsVal, batchLabels))
      print('Minibatch log loss: %.3f%%' % batchLogLossVal)
      print('Validation accuracy: %.3f%%' % accuracy(validPredsVal, validLabels))
      print('Validation log loss: %.3f%%' % validLogLossVal)
    
    if (step % 50000 == 0):
        saver.save(session, '/Users/swapnil/work/Kaggle/data/quoraPairs/models/firstModel/model',global_step=step)
    
   

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 0.678359
Minibatch accuracy: 0.600%
Minibatch log loss: 0.678%
Validation accuracy: 0.630%
Validation log loss: 0.673%
Minibatch loss at step 50: 0.627589
Minibatch accuracy: 0.700%
Minibatch log loss: 0.628%
Validation accuracy: 0.630%
Validation log loss: 0.656%
Minibatch loss at step 100: 0.669608
Minibatch accuracy: 0.600%
Minibatch log loss: 0.670%
Validation accuracy: 0.630%
Validation log loss: 0.652%
Minibatch loss at step 150: 0.697492
Minibatch accuracy: 0.540%
Minibatch log loss: 0.697%
Validation accuracy: 0.630%
Validation log loss: 0.649%
Minibatch loss at step 200: 0.593164
Minibatch accuracy: 0.720%
Minibatch log loss: 0.593%
Validation accuracy: 0.630%
Validation log loss: 0.647%


In [16]:
print(sys.getsizeof(np.zeros(11)))
print(np.shape(np.zeros(11)))
print(sys.getsizeof(validData))

print(type(batchDataFrame))
graph = tf.Graph()
tf.reset_default_graph()
with graph.as_default():
    a = tf.constant([[5,6,7],[7,8,9]])
    b = tf.Variable([[5,6,7],[7,8,9]])
    c = a*b
    r = tf.range(3)
    d = a - 1
    e = tf.reshape(a,[-1,2])

184
(11,)
24256800128
<class 'pandas.core.frame.DataFrame'>
